# Data preparation and analysis for chat model fine-tuning

https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [1]:
import json
import tiktoken     # token counting
import numpy as np 
from collections import defaultdict

## Data Loading

In [2]:
# dataset path
file_path = '../amazon-english-val-sentiment-random.jsonl'

# dataset
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

## Format validation

- Data Type check: Checks whether each entry in the dataset is a dictionary (`dict`). Errr type: `data_type`

- Presence of Message List: Checks if a `messages` list is present in each entry. Error type: `missing_message_list`

- Message Keys Check : Validates that each message in the `message` list contains the keys `role` and `content`. Error type: `message_missing_key`

- Unrecognized Keys in Messages: Logs if a message has key other than `role`, `conetent`, `weight`, `function call`, and `name`. Error type: `message_uncategorized_key`.

- Role Validation: Ensures the `role` is one of "system", "user", or "assistnat". Error type: `uncognized_role`

- Content Validation: Verifies that `content` has textual data and is a string. Error type: `missing_content`

- Assistant Message Presnece: Checks that each conversation has at least one message from the assistant. Error type: `example_missing_assistant_message`

In [3]:
# format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
    messages = ex.get("messages", None)
    if not messages:
        format_errors["messing_messages_list"] += 1
        continue
    
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
            
        if any(k not in ("role", "content", "name", "fucntion_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["uncategorized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors['missing_content'] += 1
            
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1
if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No error found")

Found errors:
messing_messages_list: 199859


## Token Couning Utilities

In [4]:
encoding = tiktoken.get_encoding('cl100k_base')

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

def num_tokens_for_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message['role'] == 'assistant':
            num_tokens += len(encoding.encode(message['content']))
    return num_tokens

def print_distribution(values, name):
    print(f"\n### Distribution of {name}:")
    print(f"min / max : {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Data Warnings and Token Counts

- Missing System / User Messages

- Number of Messages per Example

- Total Tokens per Example

- Tokens in Assistant's Message

- Token Limit Warnings

In [5]:
# warning and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_assistant_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num example  missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_message_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_token_per_example")
n_too_long = sum(1 > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

KeyError: 'messages'

# Cost Estimation

In [7]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~0 tokens that will be charged for during training
By default, you'll train for 1 epochs on this dataset
By default, you'll be charged for ~0 tokens
